In [126]:
import pandas as pd
import re

In [127]:
!git clone https://github.com/DV-11/SpanishDialectDiscrimination.git

fatal: destination path 'SpanishDialectDiscrimination' already exists and is not an empty directory.


# Loading the data

In [128]:
preseea_df = pd.read_csv('/content/SpanishDialectDiscrimination/Data/PRESEEA.csv')

In [129]:
preseea_es = preseea_df[preseea_df['country'] == 'España'].reset_index()['text']
preseea_mx = preseea_df[preseea_df['country'] == 'México'].reset_index()['text']

In [130]:
print("Numer of PS dialogues: ", len(preseea_es))
print("Numer of MS dialogues: ", len(preseea_mx))

Numer of PS dialogues:  233
Numer of MS dialogues:  90


Find specific entry index by text ID

In [131]:
text_id = 'PALM_M31CAST_016'
dataset = preseea_es

dataset.str.contains(text_id).idxmax()

230

# Parsing dialogue

In [151]:
dialogue = preseea_mx[3]


In [152]:
split_str = dialogue.split('</Trans>')
info = split_str[0]
transcript = split_str[1]

In [153]:
all = re.findall(r'(E:|I:)(.*?)(?=(?:E:|I:)|$)', transcript)

dialogue_lines = []
current_line = {'E_line': None, 'I_line': None}

for i, j in all:
    stripped_line = j.strip().removesuffix('/').strip()

    if i == 'E:':
        if current_line['I_line'] is not None:
            dialogue_lines.append(current_line)
            current_line = {'E_line': None, 'I_line': None}
        current_line['E_line'] = stripped_line
    elif i == 'I:':
        current_line['I_line'] = stripped_line

    if current_line['E_line'] is not None and current_line['I_line'] is not None:
        dialogue_lines.append(current_line)
        current_line = {'E_line': None, 'I_line': None}

if current_line['E_line'] is not None or current_line['I_line'] is not None:
    dialogue_lines.append(current_line)

In [154]:
dialogue_df = pd.DataFrame(dialogue_lines)
dialogue_df['Line_number'] = dialogue_df.index + 1
dialogue_df = dialogue_df[['Line_number', 'E_line', 'I_line']]

dialogue_df.head()

,Line_number,E_line,I_line
0,1,"<observación_complementaria = ""durante toda la...",<simultáneo> de tú </simultáneo>
1,2,¿de <simultáneo> tú? </simultáneo>,<simultáneo> de tú </simultáneo> de tú
2,3,¿usted cómo prefiere hablarle a las personas? ...,pues depende / si es mayor que yo pues le habl...
3,4,a su mamá ¿le habla de<alargamiento/> <simultá...,<simultáneo> de usted </simultáneo>
4,5,¿o de tú?,de <simultáneo> usted </simultáneo>


Clean up unnecessary tags

In [155]:
dialogue_df['E_line'] = dialogue_df['E_line'].str.replace('<alargamiento/>','')
dialogue_df['E_line'] = dialogue_df['E_line'].str.replace('<vacilación/>','')
dialogue_df['E_line'] = dialogue_df['E_line'].str.replace('</simultáneo>','')
dialogue_df['E_line'] = dialogue_df['E_line'].str.replace('<simultáneo>','')

In [156]:
dialogue_df['I_line'] = dialogue_df['I_line'].str.replace('<alargamiento/>','')
dialogue_df['I_line'] = dialogue_df['I_line'].str.replace('<vacilación/>','')
dialogue_df['I_line'] = dialogue_df['I_line'].str.replace('</simultáneo>','')
dialogue_df['I_line'] = dialogue_df['I_line'].str.replace('<simultáneo>','')

# Dialogue Info

In [157]:
tags = [i for i in info.split(' ') if '=' in i]

all_info = {}

for i in tags:
  split_tag = i.split('=')
  k = split_tag[0]
  v = split_tag[1]

  if not re.findall('"([^"]*)"',split_tag[1]):
    all_info[split_tag[0]] = None
  else:
    all_info[split_tag[0]] = re.findall('"([^"]*)"',split_tag[1])[0]

important_keys = ['audio_filename','clave_texto', 'pais', 'fecha_grab', 'fecha_trans']

dialogue_info = {key: all_info[key] for key in important_keys}

In [158]:
info

'<Trans audio_filename="GUAD_H21_060.MP3" xml:lang="español"> <Datos clave_texto=" GUAD_H21_060" tipo_texto="entrevista_semidirigida"> <Corpus corpus="PRESEEA" subcorpus="Guadalajara" ciudad="Guadalajara" pais="México"/> <Grabacion resp_grab="Jeanette Marisol Ruiz González" lugar="domicilio informante" duracion="43’19" fecha_grab="2016-09-06" sistema="MP3"/> <Transcripcion resp_trans="Jeanette Marisol Ruiz González" fecha_trans="2016-09-20" numero_palabras="9830"/> <Revision num_rev="1" resp_rev="Roxana Nayeli Reyes Cruz" fecha_rev="2016-11-24"/> <Revision num_rev="2" resp_rev="César Ignacio Prado Oliva" fecha_rev="2017-04-03"/></Datos> <Hablantes> <Hablante id="hab1" nombre="GUAD_H21_060" codigo_hab="I" sexo="hombre" grupo_edad="2" edad="44" nivel_edu="bajo" estudios="Primaria" profesion="albañil" origen="Guadalajara" papel="informante"/> <Hablante id="hab2" nombre="Jeanette Marisol Ruiz González" codigo_hab="E" sexo="mujer" grupo_edad="1" edad="29" nivel_edu="alto" estudios="licencia

In [159]:
dialogue_info

{'audio_filename': 'GUAD_H21_060.MP3',
 'clave_texto': None,
 'pais': 'México',
 'fecha_grab': '2016-09-06',
 'fecha_trans': '2016-09-20'}

# Readable transcript

In [160]:
for i in dialogue_df.iterrows():
  print(i[1].Line_number, 'E:', i[1].E_line)
  print(i[1].Line_number, 'I:', i[1].I_line)
  print('')

1 E: <observación_complementaria = "durante toda la grabación hay ruido de fondo tanto de voces como de vehículos"/> ¿cómo prefiere que le hable / de tú o de  usted? 
1 I:  de tú 

2 E: ¿de  tú? 
2 I:  de tú  de tú

3 E: ¿usted cómo prefiere hablarle a las personas? bueno / ya le dije usted ¿verdad? <risas = "I"/> <entre_risas> ¿tú </entre_risas> cómo prefieres hablarle a las personas?
3 I: pues depende / si es mayor que yo pues le hablo de usted y si es / por mi edad pues le hablo de tú

4 E: a su mamá ¿le habla de  usted? 
4 I:  de usted 

5 E: ¿o de tú?
5 I: de  usted 

6 E:  ¿de usted?  mj por respeto ¿no?
6 I: por respe <palabra_cortada/> sí aparte de que / no estamos acostumbrados a / a hablarles de usted de  tú a / a jefes de otra persona

7 E: mj  / ¿y a su papá?
7 I: también  de usted 

8 E:  no tu papá   <risas = "E"/> 
8 I:  sí 

9 E: <entre_risas> ¿ve? </entre_risas>  <risas = "E"/> 
9 I:  también  / también de usted

10 E: es que sí es un problema
10 I: sí

11 E: la verdad